In [12]:
file_path = "mcts_nn_dataset_2_Feb.npz"

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Load the dataset
with np.load("mcts_nn_dataset_2_Feb.npz") as data:
    X = data['X']
    y = data['y']

# Normalize and preprocess data
X = X.astype('float32')
y = y.astype('int32')

# Ensure model only plays +1 moves by flipping -1 moves
X[y < 0] *= -1  # Flip board for -1 moves
y = np.abs(y)   # Convert all labels to positive moves

# Convert board to 6x7x2 representation to allow overlapping boxes
X_expanded = np.zeros((X.shape[0], 6, 7, 2), dtype=np.float32)
X_expanded[..., 0] = (X == 1).astype(np.float32)  # Layer for player 1
X_expanded[..., 1] = (X == -1).astype(np.float32)  # Layer for player -1
X = X_expanded

# Split dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Optimized Transformer Model with Overlapping Boxes
def transformer_model(input_shape=(6, 7, 2), num_classes=7):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.LayerNormalization()(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.LayerNormalization()(x)

    # Reshape for Transformer Input
    x = layers.Reshape((6 * 7, 64))(x)  # Reduce feature dimensions to speed up training

    # Transformer Encoder Layers
    transformer_dim = 64  # Reduced from 512 to speed up training
    for _ in range(2):  # Reduced transformer layers
        attn_output = layers.MultiHeadAttention(num_heads=4, key_dim=transformer_dim)(x, x)  # Fewer attention heads
        attn_output = layers.Dense(transformer_dim)(attn_output)  # Ensure same shape after attention
        x = layers.Add()([x, attn_output])
        x = layers.LayerNormalization()(x)
        feed_forward = layers.Dense(transformer_dim, activation='relu')(x)  # Match transformer_dim instead of *2
        x = layers.Add()([x, feed_forward])
        x = layers.LayerNormalization()(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)  # Reduced FC layer size
    x = layers.Dropout(0.2)(x)

    # Fully connected output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model

# Build the optimized model
model = transformer_model()

# Compile model with learning rate decay
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001, decay_steps=5000, decay_rate=0.9)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=20, batch_size=500)  # Increased batch size for faster training

# Evaluate test accuracy
test_loss, test_accuracy = model.evaluate(X_val, y_val)
print(f'Test Accuracy: {test_accuracy:.4f}')

model.save("MYGTransformer.keras")



Epoch 1/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1125s 1s/step - accuracy: 0.2494 - loss: 1.7964 - val_accuracy: 0.3577 - val_loss: 1.5412
Epoch 2/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1191s 2s/step - accuracy: 0.3645 - loss: 1.5369 - val_accuracy: 0.3863 - val_loss: 1.5030
Epoch 3/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1113s 1s/step - accuracy: 0.3941 - loss: 1.4891 - val_accuracy: 0.4116 - val_loss: 1.4573
Epoch 4/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1209s 2s/step - accuracy: 0.4170 - loss: 1.4495 - val_accuracy: 0.4230 - val_loss: 1.4336
Epoch 5/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1095s 1s/step - accuracy: 0.4287 - loss: 1.4271 - val_accuracy: 0.4307 - val_loss: 1.4205
Epoch 6/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1058s 1s/step - accuracy: 0.4375 - loss: 1.4063 - val_accuracy: 0.4390 - val_loss: 1.4049
Epoch 7/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1056s 1s/step - accuracy: 0.4446 - loss: 1.3928 - val_accuracy: 0.4423 - val_loss: 1.3941
Epoch 8/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 1063s 1s/step - accuracy: 0.4520 - loss: 1.3787 - 

In [12]:
# MCTS Bot Integration

def update_board(board_temp, color, column):
    board = board_temp.copy()
    colsum = sum(abs(board[i, column]) for i in range(6))
    row = int(5 - colsum)
    if row >= 0:
        board[row, column] = 1 if color == 'plus' else -1
    return board


def check_for_win(board):
    for col in range(7):
        for row in reversed(range(6)):
            if abs(board[row, col]) < 0.1:
                break
            if row <= 2:
                if sum(board[row + i, col] for i in range(4)) == 4:
                    return 'v-plus'
                if sum(board[row + i, col] for i in range(4)) == -4:
                    return 'v-minus'
    return 'nobody'

#updated play vs mcts
def play_vs_mcts(model, num_games=10):
    wins, losses = 0, 0
    for game in range(num_games):
        board = np.zeros((6, 7), dtype=np.float32)
        print(f"\n Starting Game {game + 1}")

        for turn in range(42):
            player = 'plus' if turn % 2 == 0 else 'minus'
            
            # Expand board correctly
            board_input = np.expand_dims(board, axis=0)  
            board_input = np.expand_dims(board_input, axis=-1)
            board_input = np.concatenate([board_input == 1, board_input == -1], axis=-1).astype(np.float32)
            
            move_probs = model.predict(board_input)[0]
            move = np.argmax(move_probs)  # Pick best move
            
            print(f"Move {turn+1}: Player {player} chooses column {move}")

            board = update_board(board, player, move)
            winner = check_for_win(board)

            if winner == 'v-plus':
                wins += 1
                print(f"Model won Game {game + 1}")
                break
            elif winner == 'v-minus':
                losses += 1
                print(f" Model lost Game {game + 1}")
                break

    print(f"\n Model won {wins}/{num_games} games vs. MCTS bot")
